In [2]:
from ser.feature_extract import extract_features_from
from ser.Yoon_model import YoonBREAudio, YoonBREText
import torch
from torchtext import data
import numpy as np
from nltk.tokenize import TweetTokenizer

label_dict = {
    "neu": 0,
    "ang": 1,
    "sad": 2,
    'hap': 3
}
MAX_TEXT_LEN = 128

vector_path = 'D:/DoobiePJ/2020Ali/SER_test/.vector_cache/'
x_t = np.load('D:\DoobiePJ/2020Ali\demo\ser/x_text.npy', allow_pickle=True)
TEXT = data.Field()
TEXT.build_vocab(x_t, max_size=25000, vectors='glove.6B.100d', vectors_cache=vector_path, unk_init=torch.Tensor.normal_)


tknzr = TweetTokenizer()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions

model_a = YoonBREAudio(num_class=4, mfcc_size=119, prosody_size=35, hidden_size=128, num_layers=2, dropout=0.3)
model_a = torch.load('D:\DoobiePJ/2020Ali\demo\ser/Yoon_audio.pth')
model_t = YoonBREText(num_class=4, vocab_size=len(TEXT.vocab), embedding_dim=100,
                      pad_idx=TEXT.vocab.stoi[TEXT.pad_token], hidden_size=128, num_layers=2, dropout=.3,
                      bidirectional=True, use_cuda=True)
model_t = torch.load('D:\DoobiePJ/2020Ali\demo\ser/Yoon_text.pth')


def ser(wav_file):
    audio_feature = extract_features_from(wav_file)
    print(audio_feature)
    prediction_a = model_a(audio_feature)
    return prediction_a


def ter(text):
    tokens = tknzr.tokenize(text)
    if len(tokens) > MAX_TEXT_LEN:
        tokens = tokens[:MAX_TEXT_LEN]
    else:
        tokens = tokens + [TEXT.pad_token] * (MAX_TEXT_LEN - len(tokens))
    idxs = []
    for token in tokens:
        idxs.append(TEXT.vocab.stoi[str(token)])
    t = torch.tensor(idxs).cuda()
    t = t.view(1, t.shape[0])
    prediction_a = model_t(t)[0]
    for i, num in enumerate(label_dict):
        print("%s: %f" % (num, prediction_a[i]))
    return prediction_a

1.6.0+cu101


In [5]:
ang_text = [
    "What the hell?",
    "I hate you!",
    "I am angry. I'm like a large tornado of anger, swirling about.",
    "Yes, General. I do. You're the man responsible for the deaths of two of my men. Now, open up those crates before I grab you by your hair plugs and bitch slap that smug look off of your face.",
]

sad_text = [
    "I don't feel good. I think I'm crying.",
    "I can't stand living without you for one second. I'm not happy about that.",
    "I'm poor, Black, I might even be ugly, but dear god, I'm here. I'm here.",
    "When you burst out crying alone and u realize that no one truly knows how unhappy you really are because you don't want anyone to know"
]

happy_text = [
    "Today was perfect.",
    "Can't believe that I got the first prize. What a luck!",
    "one thing I can say is that you kept me smiling",
    "I'm so excited to see Nat tonight And how happy and cheery she is! then I'm even more excited for her to get on social media",
    "you know it's beauty when the smile is her best curve",
    "I'm still laughing 'Bitch took my pillow' line.",
    "Good morning. Let's start with a smile! Let's enjoy life in a cheerful way! Don't worry be happy!",
    "Nothing lovelier than your great mate sending the most joyful and sweet pic."
]

# for sentence in ang_text:
#     print(ter(sentence))
#
# for sentence in sad_text:
#     print(ter(sentence))

for sentence in happy_text:
    print(ter(sentence))

neu: 0.000001
ang: 0.000000
sad: 0.000003
hap: 0.999996
tensor([5.6480e-07, 6.7609e-09, 3.1896e-06, 1.0000e+00], device='cuda:0',
       grad_fn=<SelectBackward>)
neu: 0.000000
ang: 0.000002
sad: 0.000003
hap: 0.999995
tensor([8.6951e-08, 2.0325e-06, 2.7256e-06, 1.0000e+00], device='cuda:0',
       grad_fn=<SelectBackward>)
neu: 0.000000
ang: 0.999922
sad: 0.000076
hap: 0.000001
tensor([1.3301e-07, 9.9992e-01, 7.6365e-05, 1.4830e-06], device='cuda:0',
       grad_fn=<SelectBackward>)
neu: 0.000000
ang: 0.000000
sad: 0.000000
hap: 1.000000
tensor([7.0751e-09, 4.2373e-09, 2.8449e-08, 1.0000e+00], device='cuda:0',
       grad_fn=<SelectBackward>)
neu: 0.951074
ang: 0.000061
sad: 0.002022
hap: 0.046843
tensor([9.5107e-01, 6.0861e-05, 2.0224e-03, 4.6843e-02], device='cuda:0',
       grad_fn=<SelectBackward>)
neu: 0.000018
ang: 0.994746
sad: 0.000024
hap: 0.005212
tensor([1.8262e-05, 9.9475e-01, 2.4058e-05, 5.2120e-03], device='cuda:0',
       grad_fn=<SelectBackward>)
neu: 0.000027
ang: 0.0